<a href="https://colab.research.google.com/github/luimont/tesis-lcc/blob/main/Notebooks/LIWC_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LIWC Classification**

**Notebook de Pruebas con LIWC**

En esta notebook se realizan pruebas con vectores de LIWC obtenidos desde los datos reales de estudio.

### Representacion de BOW

In [54]:
# General Libraries
%matplotlib inline
import pandas as pd
import numpy as np
import os                # para manipulacion de directorios
import shutil            # para eliminar carpetas no vacias
#!pip install mglearn    # el comando 'import mglearn' no funcionaba

# Preprocessing
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

# CountVectorizer | TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Cross Validation
from sklearn.model_selection import cross_val_score

# GridSearch
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# LogisticRegression Classifier
from sklearn.linear_model import LogisticRegression

# SVM Classifier
from sklearn import svm

# KNN Clasificator
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler

# Gaussian NaiveBayes Clasificator
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

# RandomForest
from sklearn.ensemble import RandomForestClassifier

# DecisionTree (Classifier and Regressor)
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

# Graphics library
import matplotlib.pyplot as plt



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [55]:
#path_colab  = "https://drive.google.com/file/d/1ibZ-jIiQf5af18V6emfq6U3Dg6yq9Ij3/view" #"./90FeaturesLIWC.csv",
path_colab  = "https://drive.google.com/file/d/1U-3dNmKGQ-MUrt7nHbyf_hbF6aZCP5LW/view" #"./datasets_full/ID-90FeatureLIWC_Labels.csv",


# With ONLY URL
if path_colab.startswith('http'):
  print("Ubication WEB")
  path_colab='https://drive.google.com/uc?id=' + path_colab.split('/')[-2]
else: print("Ubication Local")

Ubication WEB


#### Lectura del DataFrame desde el CSV

Selección de las columnas 7 hasta el final (corresponde a todas las feature o características de Liwc)

[Visualizar archivo CSV para ver las columnas]

In [56]:
df_ = pd.read_csv(
    #'ID-90FeatureLIWC_Labels.csv',
    path_colab,
    encoding = "utf-8",
    header = 0,
    # names = ["ID","NeuroPerc", "Autodescripción"],
)

# Labels (All Personalities)
#df_labels = df_.iloc[:,1:6]

# Labels (only "AGRADO")
#colum_title = ["AgradPerc", "ApertPerc", "ExtravPerc", "NeurPerc", "RespPerc"]
#POS = 0

#df_labels = list(df_[colum_title[POS]])
#df_labels = list(df_["AgradPerc"])
## Labels (only "AGRADO")
#df_labels = list(df_["AgradPerc"])
## Labels (only "AGRADO")
#df_labels = list(df_["AgradPerc"])
## Labels (only "AGRADO")
#df_labels = list(df_["AgradPerc"])
## Labels (only "AGRADO")
#df_labels = list(df_["AgradPerc"])


# X_Train (corpus-matrix-liwc)
df_features = df_.iloc[:,7:] # Select From column numer 7 to end
#print(df2[0:3])
#print(df2.to_numpy())


#df2.index.to_numpy()
#print(df2[0:3])
# array(['a', 'b', 'c'], dtype=object)

Seleccionar la personalidad editando el valor de la variable POS

In [110]:
# Select the Personality

# 0: Agrado
# 1: Apertura
# 2: Extraversion
# 3: Neuroticismo
# 4: Responsabilidad

POS = 4

personality = ["Agrado", "Apertura", "Extraversion", "Neuroticismo", "Responsabilidad"]
colum_title = ["AgradPerc", "ApertPerc", "ExtravPerc", "NeurPerc", "RespPerc"]


# Selection from personality labels
df_labels = list(df_[colum_title[POS]])

print(f"Personality Selected: {personality[POS]}")
#print("DataFrame\n----------\n", df_labels)
#print('DataFrame\n----------\n', df_features)
#print('\nDataFrame datatypes :\n', df_features.dtypes)

Personality Selected: Responsabilidad


Muestra de los primeros 2 elementos de vector_array y vector_label

In [111]:
#convert pandas dataframe to numpy array
vector_array = df_features.to_numpy()
print('\nNumpy Array\n----------\n', vector_array[0:2])
print('\nNumpy Array Datatype :', vector_array.dtype)


#convert pandas dataframe to numpy array
vector_label = df_labels#.to_numpy()
print('\nNumpy Array\n----------\n', vector_label[0:2])

# Convert entire DataFrame string to float
# df3 = df2.astype(float)
# print(df2.dtypes)

# df2['WC'].to_numpy()
# print(df2[0:3])
# array([1, 2, 3])

#df_.as_matrix( columns=[df_[8:10]] )
#Suffle the Data
#df=df_.sample(frac=1)
#
#texts_unprocessed = list(df["Autodescripción"])   #será el X_train o simplemente X es decir, los datos, en cross_val_score function
#labels = list(df[colum_labels])               #será el y_train o simplemente y es decir, los labels, en cross_val_score function que usará para evaluar el clasificador
#print("Textos:   ", texts_unprocessed[0:3])
#print("Etiquetas:", labels[0:3])


Numpy Array
----------
 [[7.350e+02 3.196e+01 1.823e+01 9.524e+01 5.320e+01 1.946e+01 1.211e+01
  7.890e+00 2.700e-01 1.400e-01 4.080e+00 8.200e-01 7.350e+00 6.260e+00
  1.578e+01 1.630e+00 2.310e+00 1.347e+01 0.000e+00 8.980e+00 1.075e+01
  7.760e+00 1.630e+00 3.810e+00 1.360e+00 2.700e-01 4.760e+00 0.000e+00
  4.100e-01 2.720e+00 4.100e-01 1.220e+00 1.400e-01 0.000e+00 0.000e+00
  0.000e+00 1.102e+01 2.860e+00 1.500e+00 2.310e+00 7.890e+00 5.170e+00
  2.860e+00 0.000e+00 2.180e+00 2.700e-01 3.020e+01 4.220e+00 2.450e+00
  1.500e+00 4.220e+00 3.130e+00 5.400e-01 8.710e+00 1.900e+00 3.670e+00
  1.500e+00 2.700e-01 6.800e-01 2.310e+00 5.400e-01 8.200e-01 2.700e-01
  6.800e-01 1.497e+01 3.540e+00 4.900e+00 6.800e+00 2.310e+00 2.590e+00
  3.270e+00 1.900e+00 1.400e-01 0.000e+00 0.000e+00 5.400e-01 0.000e+00
  0.000e+00 4.900e+00 2.990e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 2.700e-01 0.000e+00 1.630e+00 0.000e+00]
 [1.237e+03 2.689e+01 1.754e+01 9.159e+01 5.190e

# Results

### Test the combinations

- With GridSearchCV, we can test the different combinations for parameters for Classifiers

In [114]:
# Classifiers

#SVM
svc = svm.SVC()
parameters_svc = [
    {'kernel': ['linear'], 'C':[ 1, 100, 1000], 'gamma': [0.001, 0.01, 0.1]},
    {'kernel': ['rbf', 'poly', 'sigmoid'], 'gamma':[1e-3, 1e-4,'scale','auto'], 'C':[1, 100, 1000]}
    #{'kernel': ['linear'], 'C':[ 10, 100, 1000, 100000], 'gamma': [0.001, 0.01, 0.1, 1 ]},
    #{'kernel': ['rbf', 'poly', 'sigmoid'], 'gamma':[1e-3, 1e-4,'scale','auto'], 'C':[1, 10, 100, 1000, 100000]}
]

#LogisticRegression
lr = LogisticRegression(solver='liblinear', random_state=2, max_iter=50000)
parameters_lr = [{'solver':['sag', 'newton-cg',  'saga', 'liblinear']}]

#KNN
knn = Pipeline([('mms', MaxAbsScaler()), ('knn', KNeighborsClassifier())])
parameters_knn = [{'knn__n_neighbors': [1, 3, 5, 7, 8, 9, 10], 'knn__weights': ['uniform', 'distance'], 'knn__leaf_size': [15, 20]}]

#NaiveBayes Gaussian
gnb = GaussianNB()
parameters_gnb = [{'var_smoothing': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15]}]

#NaiveBayes Multinomial
gnm = MultinomialNB()
parameters_gnm = [{'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'fit_prior' : [True, False] }]

#RandomForest
rf = RandomForestClassifier() #Initialize with whatever parameters you want to
parameters_rf = [{'n_estimators': [5, 10, 15, 20], 'max_depth': [2, 5, 7, 9]}]

#DecisionTreeClassifier
dtc = DecisionTreeClassifier()
parameters_dtc = [{
    'splitter': ['best', 'random'], 'criterion': ["gini", "entropy"], "max_depth": [*range(1, 10)],
    'min_samples_leaf': [*range(1, 50, 5)],#' min_impurity_decrease': [*np.linspace(0, 0.5, 20)]
    #'max_features':[ None, 'auto','sqrt','log2'],   #Added
    #'min_impurity_decrease': [None, 0.5]            #Added
}]

# #DecisionTreeRegressor
# dtr = DecisionTreeRegressor()
# parameters_dtr =[{
#     "criterion": ["squared_error", "friedman_mse", "absolute_error", "poisson"],
#     "splitter":["best","random"], "max_depth" : [1,3,5,7,9,11,12], "max_features":["auto","log2","sqrt",None], "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90],
#     "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10], "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
# }]

# Group the Classifier with the corresponding params

classifier_and_params = [
  {'cslf':dtc , 'params':parameters_dtc, 'name':'dtc'},
  {'cslf':rf , 'params':parameters_rf, 'name':'rf'},
  {'cslf':knn , 'params':parameters_knn, 'name':'knn'},
  {'cslf':svc , 'params':parameters_svc, 'name':'svc'},
  {'cslf':gnb , 'params':parameters_gnb, 'name':'gnb'},
  {'cslf':gnm , 'params':parameters_gnm, 'name':'gnm'},
  {'cslf':lr , 'params':parameters_lr, 'name':'lr'}
]

print(f"Total of Classifiers to test on GridSearch: {len(classifier_and_params)}")

Total of Classifiers to test on GridSearch: 7


In [115]:

#### GridSearch ####
# Para NB (NaiveBayes) se necesita tener X_traing.toarray(), ya que es este el tipo de valor que espera

def gridSearchProcess(classifier, params, name):
  clf = GridSearchCV(
      estimator=classifier,               # Pass the model instance for which you want to check the hyperparameters.
      param_grid=params,                  # the dictionary object that holds the hyperparameters you want to try
      scoring='f1_weighted',                 # evaluation metric that you want to use, you can simply pass a valid string/ object of evaluation metric
      cv=KFold(n_splits=5, shuffle=True, random_state=1), # number of cross-validation you have to try for each selected set of hyperparameters
      #refit=True,                          # Refit an estimator using the best found parameters on the whole dataset (default=True).
      #n_jobs=-1,                         # number of processes you wish to run in parallel for this task if it -1 it will use all available processors.
      #verbose=1,                         # you can set it to 1 to get the detailed print out while you fit the data to GridSearchCV
  )
  clf.fit(vector_array, vector_label)

  # find best model score
  best_params       = clf.best_params_
  best_model_score  = clf.score(vector_array, vector_label)

  print(f'\n******** {name} Results ********')
  #print(f'best_model_score: {round(best_model_score, 4)}')
  print(f'best_score:       {round(clf.best_score_, 4)}')
  print(f'scorer:           {clf.scoring}')
  #print(f'best_estimator:{clf.best_estimator_}')
  #print(f'best_index:       {clf.best_index_}')
  #print(f'X_traing:         {len(X_train.toarray())}')
  #print(f'n_splits:         {clf.n_splits_}')
  print(f'best_params:      {clf.best_params_}')

print(f"Selected Tokenized Representation: LIWC")
print(f"Selected Class Personality: {personality[POS]}")

for pair in classifier_and_params:
    gridSearchProcess(classifier=pair['cslf'], params=pair['params'], name=pair['name'])

Selected Tokenized Representation: LIWC
Selected Class Personality: Responsabilidad

******** dtc Results ********
best_score:       0.7211
scorer:           f1_weighted
best_params:      {'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 6, 'splitter': 'random'}

******** rf Results ********
best_score:       0.7123
scorer:           f1_weighted
best_params:      {'max_depth': 7, 'n_estimators': 5}

******** knn Results ********
best_score:       0.6901
scorer:           f1_weighted
best_params:      {'knn__leaf_size': 15, 'knn__n_neighbors': 7, 'knn__weights': 'uniform'}

******** svc Results ********
best_score:       0.7033
scorer:           f1_weighted
best_params:      {'C': 1000, 'gamma': 'scale', 'kernel': 'poly'}

******** gnb Results ********
best_score:       0.7265
scorer:           f1_weighted
best_params:      {'var_smoothing': 0.01}

******** gnm Results ********
best_score:       0.6961
scorer:           f1_weighted
best_params:      {'alpha': 0.1, 'fit_prior'